<a href="https://colab.research.google.com/github/37stu37/GNS_GAN/blob/master/ctgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**import**
---



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# import pandas as pd
from pathlib import Path

In [0]:
outputDir = Path('/content/drive/My Drive/Colab Notebooks/02_output')

In [0]:
data = pd.read_parquet(outputDir / 'smallMonteCarlo2500runs.parquet')

**Data processing**
---



In [0]:
data.reset_index(inplace=True)
data.drop(['index', 'scenario', 'pid'], axis=1, inplace=True)

In [7]:
len(data) #; data.columns

7864124

In [0]:
trainingSample = data.sample(n=70000)
discrete_columns = ['source', 'target']

In [9]:
trainingSample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 2235019 to 4295981
Data columns (total 5 columns):
source        70000 non-null int64
target        70000 non-null int64
distance      70000 non-null float64
bearing       70000 non-null float64
IgnProb_bl    70000 non-null float64
dtypes: float64(3), int64(2)
memory usage: 3.2 MB


**CTGAN**

In [0]:
%%capture
!pip install ctgan 

In [12]:
%%time
from ctgan import CTGANSynthesizer

e = 85
ctgan = CTGANSynthesizer()
ctgan.fit(trainingSample, discrete_columns, epochs=e)

Epoch 1, Loss G: 7.2030, Loss D: -0.0443
Epoch 2, Loss G: 6.7535, Loss D: -0.0179
Epoch 3, Loss G: 4.7185, Loss D: -0.0407
Epoch 4, Loss G: 0.4268, Loss D: 0.0278
Epoch 5, Loss G: -0.4192, Loss D: -0.0369
Epoch 6, Loss G: -0.6078, Loss D: 0.0071
Epoch 7, Loss G: -0.8324, Loss D: 0.0762
Epoch 8, Loss G: -0.9830, Loss D: -0.0050
Epoch 9, Loss G: -1.0644, Loss D: -0.0336
Epoch 10, Loss G: -1.1443, Loss D: 0.0433
Epoch 11, Loss G: -1.2133, Loss D: 0.0200
Epoch 12, Loss G: -1.1548, Loss D: -0.0295
Epoch 13, Loss G: -1.2031, Loss D: 0.0101
Epoch 14, Loss G: -1.2158, Loss D: 0.0078
Epoch 15, Loss G: -1.2219, Loss D: 0.0021
Epoch 16, Loss G: -1.2199, Loss D: 0.0580
Epoch 17, Loss G: -1.1471, Loss D: -0.0143
Epoch 18, Loss G: -1.1540, Loss D: -0.0076
Epoch 19, Loss G: -1.1976, Loss D: -0.0014
Epoch 20, Loss G: -1.1934, Loss D: -0.0056
Epoch 21, Loss G: -1.1418, Loss D: 0.0313
Epoch 22, Loss G: -1.1192, Loss D: 0.0460
Epoch 23, Loss G: -1.0699, Loss D: 0.0052
Epoch 24, Loss G: -1.0849, Loss D: -

**save ML model**

In [0]:
MLpath = outputDir / 'ctganOutput' / f'ctganTabular_{e}epochs.pt'
# torch.save(ctgan, path)
torch.save(ctgan.state_dict(), MLpath)

# and to reload it 
# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH))
# model.eval()

**Generate synthetic data**

In [0]:
!pip show pandas

In [20]:
!pip install pandas==1.0.4

     |████████████████████████████████| 10.1MB 5.6MB/s 
ERROR: ctgan 0.2.1 has requirement pandas<0.26,>=0.24, but you'll have pandas 1.0.4 which is incompatible.
  Found existing installation: pandas 0.25.3
    Uninstalling pandas-0.25.3:
      Successfully uninstalled pandas-0.25.3


In [0]:
samples = ctgan.sample(100000)
/content/drive/My Drive/Colab Notebooks/02_output/ctganOutput

In [31]:
%%time
s = 100000
for v in range(100):
  print(v)
  samples = ctgan.sample(s)
  samples.to_parquet(outputDir / 'ctganOutput' / f'smallSyntheticCTGAN{s}samplesVersion{v}.parquet', engine='pyarrow')

88
89
90
91
92
93
94
95
96
97
98
99
CPU times: user 10min 7s, sys: 1min 28s, total: 11min 36s
Wall time: 11min 38s


In [34]:
!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.57.0-cp36-none-any.whl size=28992 sha256=d2f1392f124246b1874da449c935e8152b47b34a19a286f709236b5a41c92a37
  Stored in directory: /root/.cache/pip/wheels/74/20/b5/20964ef97be73d2c3a695c9cad7bccd96d1e3e737a8163861f
Successfully built memory-profiler


In [39]:
import glob
import os
import dask
import dask.dataframe as dd
from dask.distributed import Client
import memory_profiler
%load_ext memory_profiler

pathCTGANoutput = outputDir / 'ctganOutput'
pathShapefile = '/content/drive/My Drive/Colab Notebooks/01_githubRepo/FFE/data/shapefile'

@dask.delayed
def read_and_concatenate_parquets(prefix, path=pathCTGANoutput):
  L = []
  files = glob.glob(os.path.join(path, prefix))# output_scenario_0_step_0.parquet
  for file in files:
    print("file loaded : {}".format(file))
    pqt = dd.read_parquet(file)
    L.append(pqt)
  df = dd.concat(L)
  return df


def count_fid_occurences(df):
  count = df['source'].value_counts().compute()
  count_df = pd.DataFrame({'source': count.index, 'count': count.values})
  return count_df


def load_shapefile(file_name, minx, miny, maxx, maxy, path=pathShapefile):
    # crop data
    bbox = box(minx, miny, maxx, maxy)
    # building point dataset
    gdf_buildings = gpd.read_file(os.path.join(path,file_name), bbox=bbox)
    max_extent = gdf_buildings.total_bounds
    data_size = getsizeof(gdf_buildings) /(1024.0**3)
    print("Shapefile extent : {}".format(max_extent))
    print("Asset loaded : {}".format(len(gdf_buildings)))
    # gdf_buildings.plot(column='IgnProb_bl', cmap='hsv', legend=True)
    return gdf_buildings


def merge_coordinates_export_shape(ddf, gdf, name_output, path=pathCTGANoutput):
  gdf = gdf[['TARGET_FID', 'geometry']]
  df = pd.DataFrame(gdf)
  # ddf = ddf.compute()
  df_merge = ddf.merge(df, how='left', left_on='source', right_on='TARGET_FID')
  gdf_merge = gpd.GeoDataFrame(df_merge, geometry='geometry')
  gdf_merge.to_file(os.path.join(path, name_output + ".shp"))

  f, ax = plt.subplots(1, figsize=(12, 12))
  countPlot = gdf_merge.plot(column='count', cmap='seismic', legend=True, ax=ax)
  plt.savefig(os.path.join(path, 'smallSyntheticMonteCarlo.jpg'), dpi=300)
  plt.show()
  return gdf_merge

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [40]:
%%time
%%memit
ddf = read_and_concatenate_parquets("smallSyntheticCTGAN*")
count_df = count_fid_occurences(ddf)

peak memory: 6836.73 MiB, increment: 16.86 MiB
CPU times: user 12.1 s, sys: 1.97 s, total: 14 s
Wall time: 14.7 s


In [0]:
count_df

In [0]:
gdf = load_shapefile("buildings_raw.shp", 1748570, 5425500, 1749500, 5427600) # small
gdf_count = merge_coordinates_export_shape(count_df, gdf, "smallBurnedBuildings")

**Validation**
---



In [0]:
# from FFEpostprocessing.ipynb import load_shapefile, count_fid_occurences, merge_coordinates_export_shape # comment out some def

In [0]:
# shapefile = load_shapefile("buildings_raw.shp", 1748570, 5425500, 1749500, 5427600) # small

In [0]:
# gdf_count_GAN = merge_coordinates_export_shape(count_df, gdf, "smallBurnedBuildings")